In [ ]:
import numpy as np
import scripts
import geopandas
import matplotlib.pyplot as plt
import stoclust as sc
import matplotlib.lines as ln
from tqdm import tqdm

import plotly.graph_objects as go
import scipy.linalg as la
import os

# Overview

The technical coefficients $C_{ri,sj}$, whether they describe the GTAP 8 dataset or the null model, do not *completely* describe the model. Rather, they only describe the likelihood of sector $s,j$ spending a given dollar on sector $r,i$. To complete the model we need a monetary distribution. It is sufficient to specify the spending distribution across final demand sectors: $x_{ra}$. This may be retrieve from the GTAP 8 dataset for the empirical case; for the null model, it must be generated, and in a way such that the regional deficit is controlled. How this is done precisely is described in the supplementary material,but it depends upon the technical coefficients $C_{ri,sj}$ and their Leontief inverse $\mathbf{L} = (\mathbf{I} - \mathbf{C})^{-1}$, which have already been calculated (see `inverse.ipynb` in this repo).
The regional spending distribution $\hat{x}_r$ is determined as a Dirichlet centered on the eigenvector $\pi_{r}$ of the matrix
$$
K_{rs} = \sum_{i,j,a} U_{ri}(\mathbf{I}-\mathbf{C})^{-1}_{ri,sj}c_{j,sa}s_{a,s}
$$
where $U_{ri}$ corresponds to the `FD_L` block of the $\mathbf{L}$ matrix, $c_{j,sa}$ to the `DC_L` block, and $s_{a,s}$ is itself randomly generated from a uniform Dirichlet (this quantity describes the distribution of spending among a given region's final demand sectors). With $\hat{x}_r$ and $s_{a,r}$ in hand, the demand vector $d_{ri}$ can be described as $d_{ri} = \sum_a c_{i,ra}s_{a,r}\hat{x}_r$. With $\mathbf{d}$ and $\mathbf{C}$, the attribution matrix can be readily calculated by
$$
\hat{A}_{ri,s} = \frac{\sum_j(\mathbf{I}-\mathbf{C})^{-1}_{ri,sj}d_{sj}}{\sum_{s,j}(\mathbf{I}-\mathbf{C})^{-1}_{ri,sj}d_{sj}}
$$
From the attribution matrix and an impact distribution $\hat{e}_{ri}$ we can calculate the attributed distribution $\hat{a}_s$.

To get the impact distributions for $\mathrm{CO}_2$ and labor, we draw estimated carbon production and labor-time from the GTAP 8 and GMig2 datasets. The impact distribution for the null model, termed "unobtainium," is generated according to a Dirichlet.

# Initialization

First we load in the GTAP data, the generated blocks (null and GTAP 8), and their Leontief inverses.

In [ ]:
gtap = scripts.GTAP()
num_reg = gtap.regions.items.size
num_commod = gtap.commodities.items.size
num_fac = gtap.factors.items.size

In [ ]:
new_reg = gtap.megaregions.at_scale(1)
new_commod = gtap.commodities.at_scale(2)
new_fac = gtap.factors.at_scale(2)

num_new_reg = new_reg.clusters.size
num_new_commod = new_commod.clusters.size
num_new_fac = new_fac.clusters.size

In [ ]:
names = sc.Group(['D','M','F','C'])

T_null = {
    names[i]+names[j]: np.load('scripts/data/computed/blocks/null/'+names[i]+names[j]+'_T.npy')
    for i in range(len(names))
    for j in range(len(names))
}

T_ind = {
    names[i]+names[j]: np.load('scripts/data/computed/blocks/industry/'+names[i]+names[j]+'_T.npy')
    for i in range(len(names))
    for j in range(len(names))
}

L_null = {
    names[i]+names[j]: np.load('scripts/data/computed/leontief/null/'+names[i]+names[j]+'_L.npy')
    for i in range(len(names))
    for j in range(len(names))
}

L_ind = {
    names[i]+names[j]: np.load('scripts/data/computed/leontief/industry/'+names[i]+names[j]+'_L.npy')
    for i in range(len(names))
    for j in range(len(names))
}

In [ ]:
num_samples = T_null['CC'].shape[0]

Flow_nulls = [
    [],[],[],[]
]

for k in tqdm(range(num_samples)):
    for i in range(4):
        Flow_nulls[i].append(
            sum([L_null[names[i]+n][k]@T_null[n+'C'][k] for n in names])
        )
Flow_null = {
    names[i]: np.array(Flow_nulls[i])
    for i in range(len(names))
}

In [ ]:
Flow_ind = {
    names[i]: 
    sum([L_ind[names[i]+n]@T_ind[n+'C'] for n in names])
    for i in tqdm(range(len(names)))
}

# Setting incomes

Here we generate the regional income distributions $\hat{x}_r$ for the empirical and null models, which will be necessary for calculating the attribution matrix.

In [ ]:
ind_spending = sc.utils.stoch(gtap.fetch_sam(demand=gtap.sectors['final_demand']).sum(axis=1).reshape([num_reg*3]))

ind_income = Flow_ind['F']@ind_spending

ind_activity = Flow_ind['D']@ind_spending

ind_value_added = T_ind['FD'].sum(axis=0)*ind_activity

In [ ]:
ind_nat_spending = sc.utils.stoch(gtap.fetch_sam(demand=gtap.sectors['final_demand']).sum(axis=1),axis=1).reshape([num_reg*3])
ind_flow_T = (Flow_ind['F']*ind_nat_spending[None,:]).reshape([num_reg,num_fac,num_reg,3]).sum(axis=(1,3))
eig,vec = la.eig(ind_flow_T)
ind_Y0 = np.real(sc.utils.stoch(vec[:,np.argmax(np.abs(eig))]))

In [ ]:
ind_spending_reg = ind_spending.reshape([num_reg,3]).sum(axis=1)

The Dirichlet used to calculate the null model income distribution needs a scale parameter, which we get from the corresponding Kullback-Liebler divergence in the empirical model:

In [ ]:
from scipy.special import kl_div
alpha_0 = num_reg*(1/kl_div(ind_spending_reg,ind_Y0).sum())/num_new_reg

In [ ]:
alpha = np.array([alpha_0]*4)
#alpha = alpha/np.array([1,2,4,8])
null_spending = []
for k in tqdm(range(num_samples)):
    nat_spending = np.random.dirichlet((1,)*3,size=num_new_reg).reshape([num_new_reg,3])
    nat_flow = (Flow_null['F'][k].reshape([num_new_reg,num_new_fac,num_new_reg,3]).sum(axis=1)*nat_spending[None,:,:]).sum(axis=2)
    eig,vec = la.eig(nat_flow)
    Y0 = np.real(sc.utils.stoch(vec[:,np.argmax(np.abs(eig))]))
    X = np.random.dirichlet(alpha[int(k/250)]*Y0)
    null_spending.append((X[:,None]*nat_spending).reshape(num_new_reg*3))
null_spending = np.array(null_spending)
null_activity = (null_spending[:,None,:]*Flow_null['D']).sum(axis=2)
null_income = (null_spending[:,None,:]*Flow_null['F']).sum(axis=2)
null_value_added = T_null['FD'].sum(axis=1)*null_activity

In [ ]:
X = null_spending.reshape([1000,num_new_reg,3]).sum(axis=2)
Y = null_income.reshape([1000,num_new_reg,num_new_fac]).sum(axis=2)
from scipy.special import kl_div
j = 0
kl_div(X,Y)[j*200:(j+1)*200].sum()/200

With the acquired information we can calculate the attribution matrices for the null and empirical models.

In [ ]:
Attr_D_null = (Flow_null['D']*null_spending[:,None,:]/(null_activity+(null_activity==0))[:,:,None])
Attr_D_ind = Flow_ind['D']*ind_spending[None,:]/(ind_activity+(ind_activity==0))[:,None]

# Distributing Intensities

We draw up the GTAP data on labor and carbon dioxide impact distributions. These are used to derive the intensities within each region; for the null models, these intensities are held fixed to their empirical value, while the impact distributions are adjusted to match the economic activity levels generated by the null model.

In [ ]:
lab = gtap.fetch_labor().sum(axis=0)

FD_T_big = T_ind['FD'].reshape([num_reg,num_fac,num_reg*num_commod])[:,gtap.factors['Labor'].in_superset,:]
ind_income_big = ind_income.reshape([num_reg,num_fac])[:,gtap.factors['Labor'].in_superset]
lab_ind = (FD_T_big*lab[:,:,None]*ind_activity[None,None,:]/ind_income_big[:,:,None]).sum(axis=(0,1))
lab_f_ind = (lab_ind+(ind_value_added==0))/(ind_value_added+(ind_value_added==0))

lab_agg = (gtap.commodities.measure(
    gtap.megaregions.measure(
        (lab_ind).reshape([num_reg,num_commod]),
        axis=0
    )[new_reg.clusters.in_superset],
    axis=1
)[:,new_commod.clusters.in_superset]).reshape(num_new_reg*num_new_commod)

val_agg = (gtap.commodities.measure(
    gtap.megaregions.measure(
        (ind_value_added).reshape([num_reg,num_commod]),
        axis=0
    )[new_reg.clusters.in_superset],
    axis=1
)[:,new_commod.clusters.in_superset]).reshape(num_new_reg*num_new_commod)

lab_f_null = (lab_agg+(val_agg==0))/(val_agg+(val_agg==0))
lab_null = lab_f_null[None,:]*null_value_added

In [ ]:
co2_ind = gtap.fetch_carbon(demand=gtap.sectors['activities']).sum(axis=1).reshape([num_reg*num_commod])
co2_f_ind = (co2_ind+(ind_value_added==0))/(ind_value_added+(ind_value_added==0))

co2_agg = (gtap.commodities.measure(
    gtap.megaregions.measure(
        (co2_ind).reshape([num_reg,num_commod]),
        axis=0
    )[new_reg.clusters.in_superset],
    axis=1
)[:,new_commod.clusters.in_superset]).reshape(num_new_reg*num_new_commod)

co2_f_null = (co2_agg+(val_agg==0))/(val_agg+(val_agg==0))
co2_null = co2_f_null[None,:]*null_value_added

Here, we generate the null model for the distribution of "unobtainium," a hypothetical resource whose extraction has a measurable ecological impact. 

NOTE: Run the next cell to generate for a fixed heterogeneity. To vary the heterogeneity, run the one after it. Once either cell is run, run the third cell.

In [ ]:
alpha = 0.05
unb_f_ind = np.random.dirichlet((alpha,)*num_reg*num_commod,size=num_samples)

In [ ]:
param = np.array([1,2,4,8])
unb_f_inds = []
for j in tqdm(range(num_samples)):
    unb_f_inds.append(np.random.dirichlet((param[int(j/250)]*alpha,)*num_reg*num_commod,))
unb_f_ind = np.array([unb_f_inds])

In [ ]:
unb_ind = ind_value_added[None,:]*unb_f_ind

unb_agg = (gtap.commodities.measure(
    gtap.megaregions.measure(
        (unb_ind).reshape([num_samples,num_reg,num_commod]),
        axis=1
    )[:,new_reg.clusters.in_superset],
    axis=2
)[:,:,new_commod.clusters.in_superset]).reshape(num_samples,num_new_reg*num_new_commod)

unb_f_null = (unb_agg+(val_agg==0)[None,:])/(val_agg[None,:]+(val_agg[None,:]==0))
unb_null = unb_f_null[:,None,:]*null_value_added[None,:,:]

With our impact distributions set, we pass each through the attribution matrix to get the attributed impact distribution.

In [ ]:
lab_flows_ind = lab_ind[:,None]*Attr_D_ind
co2_flows_ind = co2_ind[:,None]*Attr_D_ind
val_flows_ind = ind_value_added[:,None]*Attr_D_ind

lab_flows_null = lab_null[:,:,None]*Attr_D_null[:,:,:]
co2_flows_null = co2_null[:,:,None]*Attr_D_null[:,:,:]
val_flows_null = null_value_added[:,:,None]*Attr_D_null[:,:,:]

unb_attr_null = np.zeros([num_samples,num_samples,3*num_new_reg])
for k in tqdm(range(num_samples)):
    unb_attr_null[k,:,:] += (unb_null[k,:,:,None]*Attr_D_null).sum(axis=1)

We save the data. Use the name `null_unobtainium` for the varying heterogeneity ensemble, `null_deficit` for the varying deficit ensemble, `null` for fixed parameters and `industry` for GTAP 8 data.

In [ ]:
null_type = 'null_unobtainium'

if not os.path.isdir(os.path.abspath('scripts/data/computed/flows')):
    os.mkdir(os.path.abspath('scripts/data/computed/flows'))
if not os.path.isdir(os.path.abspath('scripts/data/computed/flows/'+null_type)):
    os.mkdir(os.path.abspath('scripts/data/computed/flows/'+null_type))
if not os.path.isdir(os.path.abspath('scripts/data/computed/flows/industry')):
    os.mkdir(os.path.abspath('scripts/data/computed/flows/industry'))

np.save('scripts/data/computed/flows/'+null_type+'/labor.npy',lab_flows_null)
np.save('scripts/data/computed/flows/'+null_type+'/carbon.npy',co2_flows_null)
np.save('scripts/data/computed/flows/'+null_type+'/values.npy',val_flows_null)
np.save('scripts/data/computed/flows/'+null_type+'/unobtainium_in.npy',unb_null)
np.save('scripts/data/computed/flows/'+null_type+'/unobtainium_out.npy',unb_attr_null)